## Example

In [1]:
import pandas as pd
import numpy as np

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

print("I_0:", I_0)
print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
for i in range(N):
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

print("C:", C)
print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

######################## PARAMTERS ENDING HERE ##################################

N: 10 T: 6
I_0: [800. 600. 425. 350. 400. 524. 453. 218. 673. 200.]
D: [[138.  55. 172. 194.  94. 185.]
 [190. 101.  68. 185.  13. 136.]
 [ 79. 179.  21.  49. 199. 200.]
 [142. 103.  78. 131. 146. 155.]
 [ 35.  62.  83.  90. 197.  49.]
 [ 91.  95. 107. 127. 116. 183.]
 [105. 164.  19. 116. 119. 175.]
 [ 37. 155.  10.  77. 168.  32.]
 [108. 185. 188. 176.  81. 172.]
 [ 46. 178. 162. 200. 154. 199.]]
I: [[  0.   0.   0.   0.   0.   0.]
 [ 48.   0.   0.   0.   0.   0.]
 [  0.  20.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 18.  23.   0.   0.   0.   0.]
 [ 28.  45.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [109.  34.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
C: {'H': array([100.,  40., 180., 180.,  40., 180., 140., 100., 180., 140.]), 'P': array([5000., 2000., 9000., 9000., 2000., 9000., 7000., 5000., 9000.,
       7000.]), 'V': array([[44., 18.,  0.],
       [89., 45.,  0.],
       [86., 38.,  0.],
       [91., 46., 

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Set parameter Username
Set parameter LicenseID to value 2631508
Academic license - for non-commercial use only - expires 2026-03-05
Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 398 rows, 264 columns and 838 nonzeros
Model fingerprint: 0xbe75289a
Variable types: 240 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 1.967757e+07
Presolve removed 365 rows and 122 columns
Presolve time: 0.00s
Presolved: 33 rows, 142 columns, 300 nonzeros
Found heuristic solution: objective 1.697771e+07
Variable types: 128 continuous, 14 integer (

In [3]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 16890367.020408165

Order quantities (x_ijt):
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[4,3,2] = 97.0
x[5,3,2] = 67.0
x[8,3,2] = 168.0
x[10,3,2] = 36.44897959183673
x[1,3,3] = 38.0
x[2,3,3] = 45.0
x[3,3,3] = 200.0
x[4,3,3] = 155.0
x[5,3,3] = 49.0
x[6,3,3] = 154.0
x[7,3,3] = 172.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 117.55102040816327
x[10,3,3] = 199.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3] = 200.0
v[1,4] = 241.0
v[2,4] = 104.0
v[3,4] = 199.0
v[4,4] = 146.0
v[5,4] = 197.0
v[6,4] = 145.0


In [4]:
#original problem, only adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = list()

for j in S_T:
    if j == 0:
        J_in_inventory.append(1)
    elif j == 1:
        J_in_inventory.append(2)
    else:
        J_in_inventory.append(3)

J_in_inventory = np.array(J_in_inventory)

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Set parameter MIPGap to value 0


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 398 rows, 264 columns and 838 nonzeros
Model fingerprint: 0xbe75289a
Variable types: 240 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 1.967757e+07
Presolve removed 365 rows and 122 columns
Presolve time: 0.00s
Presolved: 33 rows, 142 columns, 300 nonzeros
Found heuristic solution: objective 1.697771e+07
Variable types: 128 continuous, 14 integer (11 binary)

Root relaxation: objective 1.688731e+07, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

## Solve Linear relaxation

In [5]:
#linear relaxation
#also adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB


def LR(N,T,J,C,I,I_0,D,V,V_C):
    # Provided parameters (already read from the Excel file)
    # N: number of products, T: number of time periods, J: number of shipping methods
    # D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

    # Create the Gurobi model
    model = gp.Model("InventoryManagement")

    # Set error parameter
    model.setParam('MIPGap', 0.0)

    # Define sets
    S_I = range(N)  # Products i in {0,  ..., N-1}
    S_T = range(T)  # Time periods t in {0, ..., T-1}
    S_J = range(J)  # Shipping methods j in {0, ..., J-1}

    # Variables
    x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
    v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
    y = model.addVars(S_J, S_T, vtype=GRB.CONTINUOUS, name="y")  # Relaxed: Binary for shipping method y_jt
    z = model.addVars(S_T, vtype=GRB.CONTINUOUS, name="z")  # Relaxed: Number of containers z_t

    # Objective function (1)
    # Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
    holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
    purchasing_and_shipping_cost = gp.quicksum(
        (C["P"][i] + C["V"][i, j]) * x[i, j, t]
        for i in S_I for j in S_J for t in S_T
    ) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
    container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

    model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

    # Constraints
    # Inventory balance (2)
    J_in_inventory = list()

    for j in S_T:
        if j == 0:
            J_in_inventory.append(1)
        elif j == 1:
            J_in_inventory.append(2)
        else:
            J_in_inventory.append(3)

    for i in S_I:
        for t in S_T:
            # Compute the in-transit quantity arriving at time t
            in_inventory = 0
            for j in range(J_in_inventory[t]):
                in_inventory += x[i, j, t - T_lead[j] + 1]
            # Add the constraint for inventory balance
            if t == 0:
                model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            else:
                model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
                model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

    # Relate order quantity and shipping method (4)
    M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
    for j in S_J:
        for t in S_T:
            model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

    # Container constraint (5)
    for t in S_T:
        model.addConstr(
            gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
            name=f"Container_{t}"
        )

    # Non-negativity and binary constraints (6)
    for i in S_I:
        for j in S_J:
            for t in S_T:
                model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
    for i in S_I:
        for t in S_T:
            model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
    for j in S_J:
        for t in S_T:
            model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
            model.addConstr(y[j, t] <= 1, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
    for t in S_T:
        model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

    # Optimize the model
    model.optimize()
    return(model.objVal)

## Solve Naive

In [6]:
def naive(N,T,J,C,I,I_0,D,V,V_C):
    holding_cost = 0
    purchasing_and_shipping_cost = 0
    
    #purchasing and shipping cost calculation(purchase for next month's demand, use express only)
    for t in range(T - 1):
        monthly_total = 0
        for i in range(N):
            purchase = D[i, t + 1]
            purchasing_and_shipping_cost += (C["P"][i] + C["V"][i, 0]) * purchase
            monthly_total += purchase
        if monthly_total > 0:
            purchasing_and_shipping_cost += C["F"][0]
    

    #holding cost calculation 
    for i in range(N):
        holding_number_part = (I_0[i] - D[i,0] + I[i,0]) * T + I[i,1]*(T-1)
        holding_cost += holding_number_part * C["H"][i]
        for t in range(1,T):
            holding_cost += D[i,t] * C["H"][i]

    obj = holding_cost  + purchasing_and_shipping_cost
    return(obj)


In [7]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 16890367.020408165

Order quantities (x_ijt):
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[4,3,2] = 97.0
x[5,3,2] = 67.0
x[8,3,2] = 168.0
x[10,3,2] = 36.44897959183673
x[1,3,3] = 38.0
x[2,3,3] = 45.0
x[3,3,3] = 200.0
x[4,3,3] = 155.0
x[5,3,3] = 49.0
x[6,3,3] = 154.0
x[7,3,3] = 172.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 117.55102040816327
x[10,3,3] = 199.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3] = 200.0
v[1,4] = 241.0
v[2,4] = 104.0
v[3,4] = 199.0
v[4,4] = 146.0
v[5,4] = 197.0
v[6,4] = 145.0


## Solve heuristic

In [8]:
#Main heuristic

import numpy as np

def heuristic_shipping_optimizer(N, T, C, in_transit, inventory,  forecast, V, container_capacity):
    T_lead = [1, 2, 3]
    plan = {0: np.zeros((N, T)), 1: np.zeros((N, T)), 2: np.zeros((N, T))}
    shortages = np.zeros((N, T))
    leftover_volumes = np.zeros(T)
    full_containers = np.zeros(T)
    already_aggregated = np.zeros((N, T))
    leftover_product = np.zeros((N, T))

    holding_costs = C["H"]
    shipping_fixed = C["F"]
    shipping_variable = C["V"]
    container_cost = C["C"]
    purchasing_cost = C["P"]

    # Step 1: Calculate shortages (use only initial inventory + in_transit)
    I_remain = np.zeros((N, T + 1))
    for i in range(N):
        I_remain[i, 0] = inventory[i]  # 修正為一維初始庫存

    for t in range(T):
        for i in range(N):
            demand = forecast[i, t]

            available = I_remain[i, t]
            shortage = max(0, demand - available)
            shortages[i, t] = shortage

            next_inventory = max(0, available - demand) + in_transit[i, t]
            I_remain[i, t + 1] = next_inventory

    # Handle early-month shortages
    for t in range(T):
        for i in range(N):
            if shortages[i, t] > 0:
                if t - T_lead[1] < 0 and t - T_lead[2] < 0:
                    if t - T_lead[0] >= 0:
                        qty = shortages[i, t]
                        plan[0][i, t - T_lead[0]] += qty
                        shortages[i, t] = 0
                elif t - T_lead[1] >= 0 and t - T_lead[2] < 0:
                    qty = shortages[i, t]
                    plan[1][i, t - T_lead[1]] += qty
                    shortages[i, t] = 0

    # Step 2: Allocate full containers using ocean
    for t in range(T):
        air_cost_per_m3 = [(i, shipping_variable[i, 1] / V[i]) for i in range(N) if shortages[i, t] > 0 and t - T_lead[2] >= 0]
        air_cost_per_m3.sort(key=lambda x: x[1], reverse=True)

        vol_accum = sum(shortages[i, t] * V[i] for i, _ in air_cost_per_m3)
        full = np.floor(vol_accum / container_capacity)
        leftover_volume = vol_accum - full * container_capacity
        leftover_volumes[t] = leftover_volume
        full_containers[t] = full

        vol_used = 0
        for i, _ in air_cost_per_m3:
            qty = shortages[i, t]
            vol = qty * V[i]
            
            available_vol = full * container_capacity - vol_used
            if available_vol <= 0:
                leftover_product[i, t] = qty
                continue

            if vol <= available_vol:
                plan[2][i, t - T_lead[2]] += qty
                shortages[i, t] = 0
                vol_used += vol
            else:
                qty_fit = available_vol / V[i]
                if qty_fit > 0:
                    plan[2][i, t - T_lead[2]] += qty_fit
                    shortages[i, t] -= qty_fit
                    leftover_product[i, t] = shortages[i, t]
                    vol_used += qty_fit * V[i]

    # Step 3: Aggregate leftover if beneficial
    for t in range(T - 1):
        if leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            combined_vol = leftover_volumes[t] + leftover_volumes[t + 1]
            if combined_vol <= container_capacity:
                saved_cost = container_cost
                holding_cost = sum(holding_costs[i] * leftover_product[i, t + 1] for i in range(N))

                ocean_cost = shipping_fixed[2] + container_cost
                air_cost = sum(leftover_product[i, t] *  shipping_variable[i, 1] for i in range(N)) + sum(leftover_product[i, t+1] *  shipping_variable[i, 1] for i in range(N)) + 2*shipping_fixed[1] 

                if saved_cost > holding_cost and ocean_cost < air_cost:
                    for i in range(N):
                        leftover_product[i, t] += leftover_product[i, t + 1]
                        plan[2][i, t - T_lead[2]] += leftover_product[i, t]
                        leftover_product[i, t + 1] = 0
                    leftover_volumes[t] = combined_vol
                    leftover_volumes[t + 1] = 0
                    already_aggregated[:, t] = 1
                    full_containers[t] += 1  # aggregated container

    # Step 4: Decide how to transport leftover volumes
    for t in range(T):
        if np.all(already_aggregated[:, t] == 0) and leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            vol = leftover_volumes[t]
            containers_needed = np.ceil(vol / container_capacity)

            ocean_cost = shipping_fixed[2] + containers_needed * container_cost
            air_cost = shipping_fixed[1] + sum(shipping_variable[i, 1] * leftover_product[i, t] for i in range(N))

            method = 1 if air_cost < ocean_cost else 2
            for i in range(N):
                qty = leftover_product[i, t]
                if qty > 0 and t - T_lead[method] >= 0:
                    plan[method][i, t - T_lead[method]] += qty
                    shortages[i, t] = 0
            leftover_volumes[t] = 0

            if method == 2:
                full_containers[t] += 1  # unaggregated ocean shipment

    # Step 5: Calculate total cost
    total_holding_cost = 0
    # 計算每個月到達商品
    arrive = np.zeros((N, T + 1))
    for k in range(3):  # 三種運輸方式
        for i in range(N):
            for t in range(T):
                arrival_month = t + T_lead[k] - 1
                if arrival_month < T:
                    arrive[i, arrival_month] += plan[k][i, t]

    #計算每個月holding cost            
    end_stock = np.zeros((N, T + 1))
    for t in range(0,T): #第二個月與以後
        for i in range(N):
            if t == 0:
                end_stock[i,0] = inventory[i] - forecast[i,t] + in_transit[i,t] + arrive[i,t]
                total_holding_cost += end_stock[i,0] * C["H"][i]
            else:
                end_stock[i,t] = end_stock[i,t-1] - forecast[i,t] + in_transit[i,t] + arrive[i,t]
                total_holding_cost += end_stock[i,t] * C["H"][i]


    total_shipping_cost = 0
    total_purchasing_cost = 0
    total_container_cost = 0
    shipping_methods_used = {0: set(), 1: set(), 2: set()}

    # Express & Air: calculate purchasing and shipping
    for k in [0, 1]:
        for t in range(T):
            method_volume = sum(plan[k][i, t] for i in range(N))
            if method_volume > 0:
                shipping_methods_used[k].add(t)
            for i in range(N):
                qty = plan[k][i, t]
                total_purchasing_cost += qty * purchasing_cost[i]
                total_shipping_cost += qty * shipping_variable[i, k]

    # Ocean: only purchasing cost（
    for t in range(T):
        method_volume = sum(plan[2][i, t] for i in range(N)) 
        if method_volume > 0:
            shipping_methods_used[2].add(t)
        for i in range(N):
            qty = plan[2][i, t]
            total_purchasing_cost += qty * purchasing_cost[i]

    # Fixed cost for all shipping modes
    for k in range(3):
        total_shipping_cost += len(shipping_methods_used[k]) * shipping_fixed[k]

    # Ocean container cost
    total_container_cost += np.sum(full_containers) * container_cost

    total_cost = total_holding_cost + total_shipping_cost + total_purchasing_cost + total_container_cost

    return total_cost


## Sampling

In [9]:
import random

a = random.uniform(10, 20) # [10, 20]
a

11.560867395102992

| Scale | Number products/Periods | Container | Holding cost |
|--|---------------------------------|-----------|--------------|
| small |   10/6                              |    1375       |        1%      |
| medium |      100/20                           |      2750     |       2%       |
| large |     500/50           |       5500    |       4%       | 

In [10]:
scale_dict = {
    "small":{"N": 10, "T": 6, "C_C": 1375, "C_H": 0.01},
    "medium":{"N": 100, "T": 20, "C_C": 2750, "C_H": 0.02},
    "large":{"N": 500, "T": 50, "C_C": 5500, "C_H": 0.04}
}

#low -> small
#high -> large
 
scenerio_dict = {
    1: {"scale": "medium", "C_C": "medium", "C_H": "medium"},
    2: {"scale": "small", "C_C": "medium", "C_H": "medium"},
    3: {"scale": "large", "C_C": "medium", "C_H": "medium"},
    4: {"scale": "medium", "C_C": "small", "C_H": "medium"},
    5: {"scale": "medium", "C_C": "large", "C_H": "medium"},
    6: {"scale": "medium", "C_C": "medium", "C_H": "small"},
    7: {"scale": "medium", "C_C": "medium", "C_H": "large"}
}

In [11]:
file_path = "OR113-2_midtermProject_data.xlsx"

import time

LR_obj_list = list()
naive_obj_list = list()
heuristic_obj_list = list()
naive_optimality_gap = list()
heuristic_optimality_gap = list()
LR_computation_time_list = list()
naive_computation_time_list = list()
heuristic_computation_time_list = list()

## 7 scenarios
random.seed(42)
np.random.seed(42)
for scenerio in range(1,8): #(1,8)
    #for each scenario, generate 30 instances
    for instance in range(1, 31): #(1, 31)
        
        scaleof = scenerio_dict[scenerio]
        
        N = scale_dict[scaleof["scale"]]["N"]
        T = scale_dict[scaleof["scale"]]["T"]
        #print("N:", N, "T:", T) #print product number, period number in this scenario
        

        I = np.zeros([N, T])
        D = np.zeros([N, T])
        I_0 = np.zeros([N])
        
        for i in range(N):
            for t in range(T):
                D[i, t] = random.uniform(0, 200)
    
            I_0[i] = random.uniform(D[i, 0], 400)
        
        #print("I_0:", I_0) #initial inventory of i, not smaller than D[i,0]
        #print("D:", D) #demand D[i,t]
        
        
        # Read the Excel file from the 'In-transit' sheet ##it's needed??
        #df_in_transit = pd.read_excel(file_path, sheet_name="In-transit") 

        for i in range(N):
            for t in range(T):
                if t == 0: # first period
                    I[i, t] = random.choice([0, 1]) * random.uniform(0, 200) # correspond to I_i,1
                elif t == 1: # second period
                    I[i, t] = random.choice([0, 1]) * random.uniform(0, 50) # correspond to I_i,2                       
                else:
                    I[i, t] = 0
        
        #print("I:", I) #in transit
        
        # Read the Excel file from the 'Shipping cost' sheet ##it's needed??
        #df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost") 
        #J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
        #df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")
        
        #all costs
        C = {
            "H": np.zeros([N]),
            "P": np.zeros([N]),
            "V": np.zeros([N, J]),
            "F": np.array([100, 80, 50]), #fixed
            "C": scale_dict[scaleof["C_C"]]["C_C"], ##找到scenario對應的container cost (originally 2750)
        }
        
        V = np.zeros([N])#vi: volume of product i
        V_C = 30
        for i in range(N):
            C["P"][i] = random.uniform(1000, 10000)
            C["H"][i] = C["P"][i] * scale_dict[scaleof["C_H"]]["C_H"] ##找到scenario對應的holding cost比例 
            
            V[i] = random.uniform(0, 1)

            #variable cost of two shipping method 
            C["V"][i, 0] = random.uniform(40, 100)
            alpha = random.uniform(0.4, 0.6)
            C["V"][i, 1] = alpha* C["V"][i, 0]
            C["V"][i, 2] = 0 # No variable shipping cost for ocrean freight
        
        #print("C:", C)
        #print("V:", V)
        T_lead = np.array([1, 2, 3]) # T_j, lead time of 3 shipping methods
    
        start_time = time.time()
        LR_obj = LR(N,T,J,C,I,I_0,D,V,V_C)
        end_time = time.time()
        computation_time = end_time - start_time
        LR_computation_time_list.append(computation_time)

        LR_obj_list.append(LR_obj)

        start_time = time.time()
        naive_obj = naive(N,T,J,C,I,I_0,D,V,V_C)
        end_time = time.time()
        computation_time = end_time - start_time
        naive_computation_time_list.append(computation_time)
        naive_obj_list.append(naive_obj)

        start_time = time.time()
        heuristic_obj= heuristic_shipping_optimizer(N,T,C,I,I_0,D,V,V_C)
        end_time = time.time()
        heuristic_obj_list.append(heuristic_obj)
        computation_time = end_time - start_time
        heuristic_computation_time_list.append(computation_time)

        naive_optimality_gap.append((naive_obj - LR_obj) / LR_obj)
        heuristic_optimality_gap.append((heuristic_obj - LR_obj) / LR_obj)

Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 12120 rows, 8080 columns and 27720 nonzeros
Model fingerprint: 0xb0088b04
Coefficient statistics:
  Matrix range     [1e-02, 2e+05]
  Objective range  [2e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 6e+02]
Presolve removed 10377 rows and 937 columns
Presolve time: 0.02s
Presolved: 1743 rows, 7143 columns, 14132 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9926309e+07   2.123586e+04   0.000000e+00      0s
    3112    8.8348050e+08   0.000000e+00   0.000000e+00      0s

Solved in 3112 iterations and 0.04 seconds (0.02 work units)
Optimal objective  8.834805025e+08
Set parameter MIPGap to val

In [12]:
import numpy as np
import pandas as pd

# 假設這三個 list 長度都是 210（你可以用實際數據替換）

group_ids = []
obj_means = []
obj_stds = []
gap_means = []
gap_stds = []
time_means = []
time_stds = []

for i in range(0, 210, 30):
    group_ids.append(i // 30 + 1)

    # 每 30 筆切一段
    obj_chunk = heuristic_obj_list[i:i+30]
    gap_chunk = heuristic_optimality_gap[i:i+30]
    time_chunk = heuristic_computation_time_list[i:i+30]

    # 計算平均與標準差
    obj_means.append(np.mean(obj_chunk))
    obj_stds.append(np.std(obj_chunk))

    gap_means.append(np.mean(gap_chunk))
    gap_stds.append(np.std(gap_chunk))

    time_means.append(np.mean(time_chunk))
    time_stds.append(np.std(time_chunk))

# 建立 DataFrame
df = pd.DataFrame({
    "Group": group_ids,
    "Obj_Mean": obj_means,
    "Obj_Std": obj_stds,
    "Gap_Mean": gap_means,
    "Gap_Std": gap_stds,
    "Time_Mean": time_means,
    "Time_Std": time_stds
})

print(df)

   Group      Obj_Mean       Obj_Std  Gap_Mean   Gap_Std  Time_Mean  Time_Std
0      1  9.520777e+08  5.148783e+07  0.003090  0.000273   0.016997  0.003961
1      2  1.604982e+07  3.784430e+06  0.002366  0.001363   0.000692  0.002057
2      3  1.327587e+10  2.669903e+08  0.003149  0.000110   0.240668  0.052838
3      4  9.713038e+08  4.456181e+07  0.000345  0.000077   0.016483  0.004618
4      5  9.670631e+08  4.697281e+07  0.010311  0.000644   0.015409  0.003818
5      6  9.500611e+08  4.876476e+07  0.003097  0.000263   0.015752  0.003871
6      7  9.588104e+08  4.345699e+07  0.003099  0.000363   0.015260  0.004945


In [13]:
print(LR_obj_list)

[883480502.4856892, 932953968.0803373, 951523392.9536357, 972065358.9513379, 855316646.6274035, 1006613515.8566048, 880434149.2744402, 896988738.1964253, 950081740.07424, 948881433.3020138, 851715964.6397325, 894461862.0443537, 885012804.8950022, 995454939.5090749, 916882923.5600215, 961393803.3287277, 990571022.4472194, 934080613.8219908, 925433963.554951, 944814737.9189556, 1057302810.1385459, 1026872322.4642426, 943175655.9134741, 988902076.8647974, 988670819.6590723, 967262428.40981, 927512372.9258057, 1038680138.4870727, 960916336.4296274, 997024994.6989882, 17714480.312222876, 18785972.86158502, 16446083.913369391, 19434309.331621353, 13009922.947477685, 12814816.020436402, 15811927.31354728, 12007919.94897475, 15187287.030741025, 17745142.380872484, 21940976.329562303, 17274933.211195685, 21629361.448222343, 21513062.743168436, 20032290.75420482, 16896510.186776757, 18561517.30236766, 14364330.48288665, 20177391.29453081, 5005626.456886545, 16794294.1165882, 13643926.65127581, 1

In [14]:
print(naive_obj_list)

[1050447520.043916, 1120212140.846089, 1128761501.541832, 1156555601.9009285, 1016071020.2559074, 1191388893.945165, 1051147636.192481, 1056980937.0379848, 1120768426.5306025, 1109267251.0020947, 1010975310.8234663, 1057515086.2764957, 1030643949.3618013, 1171183193.093323, 1076063963.351465, 1132038804.9957652, 1149193724.5126634, 1098427825.9547842, 1080257070.4129448, 1095737145.12499, 1273603014.1551688, 1209928219.8992376, 1139303508.9581642, 1167953424.4147346, 1168211646.9868362, 1135608594.6268108, 1088968775.681476, 1226129706.4230404, 1118970786.594072, 1172335753.566852, 33845713.195440024, 30530117.024013475, 28351166.77526141, 34922845.0607859, 25923126.253432106, 24565526.05756448, 29641992.300871935, 31919990.629440363, 27842024.55631946, 28010826.897833794, 36373242.1824454, 26810132.12219216, 30566914.40716307, 32568623.115226217, 31507610.974830456, 23438614.877651043, 31139897.015717376, 25989032.7651876, 30988459.40727341, 16332765.745960746, 30889978.160982687, 192

In [15]:
print(naive_optimality_gap)

[0.1889877785513794, 0.20071533984796416, 0.18626773645367692, 0.18979201475569324, 0.18794720559032027, 0.18356139191248644, 0.19389693943462405, 0.17836589471932027, 0.17965473838390467, 0.16902619449719253, 0.18698645181683188, 0.18229198040872607, 0.16455258461946962, 0.17653059582075242, 0.17361108566989586, 0.17749750526391644, 0.16013258864928678, 0.17594542665898127, 0.1672978439901408, 0.15973756668789418, 0.20457734713509326, 0.1782654897112288, 0.20794414255183308, 0.18106074578951173, 0.1815981859257015, 0.1740439422357837, 0.1740746619329312, 0.18046900194799542, 0.16448305036805852, 0.17583386554997246, 0.9106241108347222, 0.6251549626393728, 0.7238855720670444, 0.796968673538777, 0.9925657022017939, 0.9169628357042859, 0.8746602936553717, 1.6582447888625151, 0.8332454308635648, 0.57850674266929, 0.6577768298048661, 0.551967338711146, 0.41321390741635966, 0.5138998804606992, 0.5728411374129512, 0.3871867396614332, 0.6776590247686943, 0.8092756078085481, 0.5358010832487052

In [16]:
print(heuristic_optimality_gap)

[0.003387324806208072, 0.0032201612563776245, 0.0028480670611654903, 0.00312498455985932, 0.003199878418055632, 0.0031752300992536424, 0.0036459662038289243, 0.0032635601269820555, 0.0032548673567799617, 0.003163377799137911, 0.0032539805617900013, 0.002674660035127168, 0.003322727002475505, 0.003395695767915113, 0.002861297355051096, 0.0026341171898362254, 0.0033279673423666197, 0.0030214198046834926, 0.0029808750439119895, 0.003672263369871886, 0.0027165398138358017, 0.0031211686003040197, 0.00261944583416623, 0.003119333309339791, 0.0029019239232693193, 0.003100197330657283, 0.0028046324606664187, 0.0028797693232023476, 0.00323911957881406, 0.002755611049979638, 0.0006808190824377099, 0.00021731813786011464, 0.001243162453631582, 0.0009370415534933709, 0.0010829695940541022, 0.003048029030383304, 0.001750204381151226, 0.002003588982321338, 0.0013778235220877784, 0.0017674772828308685, 0.0018043507533298867, 0.002982496082667534, 0.0035392803209705957, 0.0020050365792448088, 0.001507

In [17]:
print(len(naive_optimality_gap))

210


In [18]:
print(len(heuristic_optimality_gap))

210


In [19]:
scenarioID = [i for i in range(1, 8) for _ in range(30)]
instanceID = [i for _ in range(7) for i in range(1, 31)]


In [20]:
data = {"scenario ID":scenarioID,"instance ID":instanceID, "LR_obj":LR_obj_list,"heuristic_obj":heuristic_obj_list,"naive_obj":naive_obj_list,"LR_time":LR_computation_time_list,"heu_time":heuristic_computation_time_list,"naive_time":naive_computation_time_list,"heu_gap":heuristic_optimality_gap,"naive_gap":naive_optimality_gap}
df = pd.DataFrame(data)
df.head(210)

,scenario ID,instance ID,LR_obj,heuristic_obj,naive_obj,LR_time,heu_time,naive_time,heu_gap,naive_gap
0,1,1,8.834805e+08,8.864731e+08,1.050448e+09,0.405689,0.022830,0.000000,0.003387,0.188988
1,1,2,9.329540e+08,9.359582e+08,1.120212e+09,0.451425,0.019267,0.008341,0.003220,0.200715
2,1,3,9.515234e+08,9.542334e+08,1.128762e+09,0.386962,0.015100,0.004180,0.002848,0.186268
3,1,4,9.720654e+08,9.751030e+08,1.156556e+09,0.344752,0.013230,0.003841,0.003125,0.189792
4,1,5,8.553166e+08,8.580536e+08,1.016071e+09,0.382834,0.014729,0.000000,0.003200,0.187947
...,...,...,...,...,...,...,...,...,...,...
205,7,26,8.900451e+08,8.931854e+08,1.101244e+09,0.359396,0.016589,0.002005,0.003528,0.237291
206,7,27,9.570524e+08,9.597240e+08,1.170198e+09,0.404624,0.016068,0.006996,0.002791,0.222710
207,7,28,9.355261e+08,9.385550e+08,1.144139e+09,0.366781,0.018974,0.002013,0.003238,0.222990
208,7,29,9.605558e+08,9.636409e+08,1.173384e+09,0.385956,0.012352,0.009131,0.003212,0.221567


In [21]:
df.to_csv("result.csv",index=False)

## Get both solution for all instances